In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
from importlib import reload
from datetime import datetime

sys.path.insert(0, '../src')
import deepleeo.dataset.data_augment as dtaug
import deepleeo.dataset.utils as dsutils 
import deepleeo.utils.geofunctions as gf
import deepleeo.networks.model_builder as mb

reload(dtaug)
reload(dsutils)
reload(mb)
reload(gf)

/home/raian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'deepleeo.utils.geofunctions' from '../src/deepleeo/utils/geofunctions.py'>

# Load input Dataset

In [2]:
# DATA_DIR = os.path.join(os.path.abspath(os.path.dirname("__file__")), '../', 'data_real', 'generated')
network = "unet"
DATA_DIR = "/home/raian/doutorado/Dados/generated"
DATASET_FILE = os.path.join(DATA_DIR, 'dataset_286x286_timesstack-2015-2016.npz')#'dataset_1.npz')

# model_dir = os.path.join(DATA_DIR, 'tf_logs', "test_%s_%s" % (network, datetime.now().strftime('%d_%m_%Y-%H_%M_%S')))
model_dir = "/home/raian/doutorado/Dados/generated/tf_logs/test_debug"
# model_dir = os.path.join(DATA_DIR, 'tf_logs', 'test_fcn8s_11_12_2018-17_26_32')

In [3]:
!rm -rf {model_dir}/*
!ls -al {model_dir}

total 8
drwxr-xr-x  2 raian digits 4096 Mar 29 12:41 .
drwxr-xr-x 15 raian digits 4096 Feb 22 22:26 ..


In [4]:
dataset = np.load(DATASET_FILE)

print("Data Loaded:")
print("  -> Images: ", len(dataset["images"]))
print("  -> Labels: ", len(dataset["labels"]))
print("  -> Classes: ", len(dataset["classes"]))

print("Images shape: ", dataset["images"][0].shape, " - DType: ", dataset["images"][0].dtype)
print("Labels shape: ", dataset["labels"][0].shape, " - DType: ", dataset["labels"][0].dtype)
# print("UNIQUE LABELS: ", np.unique(dataset["labels"]))

Data Loaded:
  -> Images:  5000
  -> Labels:  5000
  -> Classes:  2
Images shape:  (286, 286, 10)  - DType:  float32
Labels shape:  (286, 286, 1)  - DType:  int32


# Perform Data Augmentation

In [5]:
# reload(dtaug)
# new_dataset = {}
# new_dataset["images"] = dataset["images"]
# new_dataset["labels"] = dataset["labels"]

# steps = 1000
# num_chips = len(dataset["images"])
# for i in range(0, num_chips, steps):
#     endstep = i + steps
#     if endstep > num_chips:
#         endstep = num_chips - 1
    
#     angles = [90, 180, 270]
#     rotated_imgs = dtaug.rotate_images(dataset["images"][i:endstep], angles)
#     flipped_imgs = dtaug.flip_images(dataset["images"][i:endstep])

#     new_dataset["images"] = np.concatenate((new_dataset["images"], rotated_imgs))
#     new_dataset["images"] = np.concatenate((new_dataset["images"], flipped_imgs))

#     rotated_lbls = dtaug.rotate_images(dataset["labels"][i:endstep], angles)
#     flipped_lbls = dtaug.flip_images(dataset["labels"][i:endstep])

#     new_dataset["labels"] = np.concatenate((new_dataset["labels"], rotated_lbls))
#     new_dataset["labels"] = np.concatenate((new_dataset["labels"], flipped_lbls)).astype(dtype=np.int32)

# new_dataset["classes"] = dataset["classes"]

# print("Data Augmentation Applied:")
# print("  -> Images: ", new_dataset["images"].shape)
# print("  -> Labels: ", new_dataset["labels"].shape)

# Split dataset between train, test and validation data

In [6]:
train_images, test_images, valid_images, train_labels, test_labels, valid_labels = dsutils.split_dataset(dataset)

print("Splitted dataset:")
print("  -> Train images: ", train_images.shape)
print("  -> Test images: ", test_images.shape)
print("  -> Validation images: ", valid_images.shape)
print("  -> Train Labels: ", train_labels.shape)
print("  -> Test Labels: ", test_labels.shape)
print("  -> Validation Labels: ", valid_labels.shape)

Splitted dataset:
  -> Train images:  (3500, 286, 286, 10)
  -> Test images:  (1500, 286, 286, 10)
  -> Validation images:  (0, 286, 286, 10)
  -> Train Labels:  (3500, 286, 286, 1)
  -> Test Labels:  (1500, 286, 286, 1)
  -> Validation Labels:  (0, 286, 286, 1)


# Train the Network

In [7]:
params = {
    "epochs": 20,
    "batch_size": 100,
    "learning_rate": 0.0001,
    "l2_reg_rate": 0.5,
    "var_scale_factor": 2.0,
    "chips_tensorboard": 2,
    "dropout_rate": 0.5,
    "num_classes": len(dataset["classes"]),
    "bands_plot": [1,2,3]
}

In [ ]:
model = mb.ModelBuilder(network)
model.train(train_images, test_images, train_labels, test_labels, params, model_dir)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `tf.contrib.distribute.MirroredStrategy` instead.
INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_model_dir': '/home/raian/doutorado/Dados/generated/tf_logs/test_debug', '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_device_fn': None, '_task_type': 'worker', '_protocol': None, '_

INFO:tensorflow:accuracy = 0.0, cross_entropy = 0.6739956, f1_score = 0.6709023, loss = 0.41648543, mean_iou = 0.0 (25.137 sec)
INFO:tensorflow:Saving checkpoints for 35 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:Loss for final step: 0.3829681.
---------------
Evaluating...
INFO:tensorflow:Calling model_fn.
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10

SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
INFO:tensorflow:Done c

INFO:tensorflow:Finished evaluation at 2019-03-29-15:49:07
INFO:tensorflow:Saving dict for global step 140: eval_metrics/accuracy = 3.0666667e-06, eval_metrics/cross_entropy = 0.47041315, eval_metrics/f1-score = 0.88532776, eval_metrics/mean_iou = 0.66398734, global_step = 140, loss = 0.28996557
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 140: /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-140
Epoch  5
---------------
Training...
INFO:tensorflow:Calling model_fn.
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)


SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-29T15:51:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-210
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow

...

SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-385
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoi

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 455: /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-455
Epoch  14
---------------
Training...
INFO:tensorflow:Calling model_fn.
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SH

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-29T16:03:34Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-525
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:accuracy = 0.000832, cross_entropy = 0.37182114, f1_score = 0.95985335, loss = 0.181975, mean_iou = 0.0
INFO:tensorflow:Finished evaluation at 2019-03-29-16:03:45
INFO:tensorflow:Saving dict for global step 525: eval_metrics/accuracy = 0.0010133333, eval_metrics/cross_entropy = 0.3707558, eval_metrics/f1-score = 0.9613106, eval_metrics/mean_iou = 0.79877794, global_step = 525, loss = 0.17217487
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 525: /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-525
Epoch  16
---------------
Training...
INFO:tensorflow:Calling model_fn.
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE 

SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-29T16:06:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt-595
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow

INFO:tensorflow:Saving checkpoints for 665 into /home/raian/doutorado/Dados/generated/tf_logs/test_debug/model.ckpt.
INFO:tensorflow:Loss for final step: 0.07586841.
---------------
Evaluating...
INFO:tensorflow:Calling model_fn.
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?, 68, 68, 128)
SHAPE Conv_3:  (?, 66, 66, 256)
SHAPE Pool_3:  (?, 32, 32, 256)
SHAPE Conv_4:  (?, 30, 30, 512)
SHAPE Pool_4:  (?, 14, 14, 512)
SHAPE Conv_5:  (?, 12, 12, 1024)
SHAPE Pool_5:  (?, 10, 10, 1024)
SHAPE Conv_1:  (?, 284, 284, 64)
SHAPE Pool_1:  (?, 141, 141, 64)
SHAPE Conv_2:  (?, 139, 139, 128)
SHAPE Pool_2:  (?,

In [ ]:
#fcn.fcn_evaluate(valid_images, valid_labels,)